In [16]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
filepath_dict = {'yelp':   'sentiment-labelled-sentences/yelp_labelled.txt',
                 'amazon': 'sentiment-labelled-sentences/amazon_cells_labelled.txt',
                 'imdb':   'sentiment-labelled-sentences/imdb_labelled.txt'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source  # Add another column filled with the source name
    df_list.append(df)

df = pd.concat(df_list)
df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [6]:
sentences = ['John likes ice cream', 'John hates chocolate.']

In [8]:
vectorizer = CountVectorizer(lowercase=False, min_df=1)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'John': 0, 'likes': 5, 'ice': 4, 'cream': 2, 'hates': 3, 'chocolate': 1}

In [9]:
vectorizer.transform(sentences)

<2x6 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [10]:
vectorizer.transform(sentences).toarray()

array([[1, 0, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0]])

In [11]:
def report(vectorizer, sentences):

    vectorizer.fit(sentences)


    items = vectorizer.vocabulary_.items()

    matrix = vectorizer.transform(sentences)

    transformed_array = matrix.toarray()

    for i, sentence in enumerate(sentences):
        print("\nSentence:", sentence, "\n")
        transformed_sentence = transformed_array[i]
        for index, value in enumerate(transformed_sentence):
            for item in items:
                if item[1] == index:
                    if value:
                        print("\t", item[0], ": found")
                    else:
                        print("\t", item[0], ": not found")
                        

report(CountVectorizer(), sentences)


Sentence: John likes ice cream 

	 chocolate : not found
	 cream : found
	 hates : not found
	 ice : found
	 john : found
	 likes : found

Sentence: John hates chocolate. 

	 chocolate : found
	 cream : not found
	 hates : found
	 ice : not found
	 john : found
	 likes : not found


In [13]:
df_yelp = df[df['source'] == 'yelp']

sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

In [14]:
len(sentences_train)


750

In [15]:
sentences_train[:4]

array(['The food was barely lukewarm, so it must have been sitting waiting for the server to bring it out to us.',
       'Sorry, I will not be getting food from here anytime soon :(',
       'Of all the dishes, the salmon was the best, but all were great.',
       'The fries were not hot, and neither was my burger.'], dtype=object)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_train

<750x1714 sparse matrix of type '<class 'numpy.int64'>'
	with 7368 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.796


In [19]:
for source in df['source'].unique():
    df_source = df[df['source'] == source]
    sentences = df_source['sentence'].values
    y = df_source['label'].values

    sentences_train, sentences_test, y_train, y_test = train_test_split(
        sentences, y, test_size=0.25, random_state=1000)

    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    X_test  = vectorizer.transform(sentences_test)

    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    score = classifier.score(X_test, y_test)
    print('Accuracy for {} data: {:.4f}'.format(source, score))

Accuracy for yelp data: 0.7960
Accuracy for amazon data: 0.7960
Accuracy for imdb data: 0.7487
